In [1]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
import requests



In [2]:
# Fetch real economic data from FRED API (Replace 'YOUR_API_KEY' with a valid FRED API key)
FRED_API_KEY = "df0d5f49a04e21c8b62b0798526f5172"
BASE_URL = "https://api.stlouisfed.org/fred/series/observations"

def fetch_fred_data(series_id, start_year="2015", end_year="2023"):
    url = f"{BASE_URL}?series_id={series_id}&api_key={FRED_API_KEY}&file_type=json&observation_start={start_year}-01-01&observation_end={end_year}-12-31"
    response = requests.get(url)
    data = response.json().get("observations", [])
    return pd.DataFrame([(obs["date"], float(obs["value"])) for obs in data], columns=["Year", series_id]) if data else pd.DataFrame(columns=["Year", series_id])



In [3]:
# Fetch data for GDP Growth, Inflation, Unemployment, and Interest Rates
gdp_df = fetch_fred_data("A191RL1Q225SBEA")  # GDP Growth Rate
inflation_df = fetch_fred_data("CPIAUCSL")  # Inflation Rate (Consumer Price Index)
unemployment_df = fetch_fred_data("UNRATE")  # Unemployment Rate
interest_df = fetch_fred_data("FEDFUNDS")  # Interest Rate
cpi_df = fetch_fred_data("CPIAUCSL") # Consumer Price Index (CPI)
pce_df = fetch_fred_data("PCE") # Personal Consumption Tax (PCE)

In [4]:
# Convert Year to datetime for merging
gdp_df["Year"] = pd.to_datetime(gdp_df["Year"]).dt.year
inflation_df["Year"] = pd.to_datetime(inflation_df["Year"]).dt.year
unemployment_df["Year"] = pd.to_datetime(unemployment_df["Year"]).dt.year
interest_df["Year"] = pd.to_datetime(interest_df["Year"]).dt.year
cpi_df["Year"] = pd.to_datetime(cpi_df["Year"]).dt.year
pce_df["Year"] = pd.to_datetime(pce_df["Year"]).dt.year

In [5]:
# Merge data into a single DataFrame
df = gdp_df.rename(columns={"A191RL1Q225SBEA": "GDP Growth Rate"})
for new_df, col_name in zip([inflation_df, unemployment_df, interest_df, cpi_df, pce_df], ["Inflation Rate", "Unemployment Rate", "Interest Rate", "CPI", "PCE"]):
    if not new_df.empty:
        df = df.merge(new_df, on="Year", how="outer").rename(columns={new_df.columns[1]: col_name})



In [6]:
# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the dashboard
app.layout = html.Div([
    html.H1("Economic Indicators Dashboard"),
    
    dcc.Dropdown(
        id='indicator-dropdown',
        options=[
            {'label': 'GDP Growth Rate', 'value': 'GDP Growth Rate'},
            {'label': 'Inflation Rate', 'value': 'Inflation Rate'},
            {'label': 'Unemployment Rate', 'value': 'Unemployment Rate'},
            {'label': 'Interest Rate', 'value': 'Interest Rate'},
            {'label': 'CPI', 'value': 'CPI'},
            {'label': 'PCE', 'value': 'PCE'}
        ],
        value='GDP Growth Rate',
        clearable=False
    ),
    
    dcc.Graph(id='indicator-graph')
])


In [7]:

# Callback to update the graph based on dropdown selection
@app.callback(
    dash.Output('indicator-graph', 'figure'),
    [dash.Input('indicator-dropdown', 'value')]
)
def update_graph(selected_indicator):
    if selected_indicator in df.columns:
        fig = px.line(df, x='Year', y=selected_indicator, title=f'{selected_indicator} Over Time')
    else:
        fig = px.line(title=f'No Data Available for {selected_indicator}')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)